In [22]:
#library imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
#Load dataset
data=pd.read_csv("California_Houses.csv")
X = data.drop('Median_House_Value', axis=1)
T= data['Median_House_Value']

#Split dataset
X_train, X_temp, T_train, T_temp = train_test_split(
    X, T, test_size=0.30, random_state=42
)

X_test, X_val, T_test, T_val = train_test_split(
    X_temp, T_temp, test_size=0.5, random_state=42
)
len(X_train)

14448

In [24]:
print("Training set:", len(X_train))
print("Validation set:", len(X_val))
print("Test set:", len(X_test))
print("Total:", len(X_train) + len(X_val) + len(X_test))

Training set: 14448
Validation set: 3096
Test set: 3096
Total: 20640


In [25]:
X_design_train = np.c_[np.ones((X_train.shape[0], 1)), X_train.values]
T_train_vec = T_train.values.reshape(-1, 1)
XtX=X_design_train.T@X_design_train
XtX_inv=np.linalg.inv(XtX)
#Calculating weights
w=XtX_inv@X_design_train.T@T_train_vec


In [ ]:
#Testing
from sklearn.metrics import mean_squared_error, r2_score
X_design_test = np.c_[np.ones((X_test.shape[0], 1)), X_test.values]

y_pred_test = X_design_test @ w  
y_test_vec = T_test.values.reshape(-1, 1)

mse_test = mean_squared_error(y_test_vec, y_pred_test)
rmse_test = np.sqrt(mse_test)
r2_test = r2_score(y_test_vec, y_pred_test)

print("Test MSE:", mse_test)
print("Test RMSE:", rmse_test)
print("Test R²:", r2_test)





Test MSE: 4907211997.419816
Test RMSE: 70051.49532608005
Test R²: 0.6233241175910397
